In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import csv
import sys
import os
import io 
import codecs
import re, string, random
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
from nltk.corpus import subjectivity
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import FreqDist, classify, NaiveBayesClassifier
import pickle
# nltk.download('punkt')
# nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|[(/)+]'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token
            
def get_reviews_for_model(cleaned_tokens_list):
    for review_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in review_tokens)
        
if __name__ == "__main__":
    
    review_data = []
    stop_words = stopwords.words('english')
    data = pd.read_csv('Data_Files/dataset.csv')
   
    for index, row in data.iterrows():
        review_data.append(row['review'])
    
    clean_token = []
    for r in review_data:
        token_data = word_tokenize(r)
        clean_token.append(remove_noise(token_data, stop_words))
    
    all_pos_words = get_all_words(clean_token)
    
    frequency_distribute = FreqDist(all_pos_words)
    
    dataset = []
    count = 0
    total_tokens_for_model = get_reviews_for_model(clean_token)
    
    for j in total_tokens_for_model:
        new_token = (j, data['sentiment'][count])
        dataset.append(new_token)
        count+=1

    train_data = dataset[:35000]
    
    test_data = dataset[35000:]
    
    classifier = NaiveBayesClassifier.train(train_data)
    
    saved_weights = open('Model_Save/Train_File/Sentiment_Classifier.pickle', 'wb')
    pickle.dump(classifier, saved_weights)
    saved_weights.close()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hp/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\hp/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
with open('Model_Save/Train_File/Sentiment_Classifier.pickle', 'rb') as cfile:
    Naive_Classifier = pickle.load(cfile)

Naive_Classifier.train(train_data)
# print("Accuracy is:", classify.accuracy(Naive_Classifier, test_data))

custom_review = "good looking movie heroes such a amazing talented i have not seen ever but background image not a relible view but emotionalable movie."

custom_tokens = remove_noise(word_tokenize(custom_review))

print(Naive_Classifier.classify(dict([token, True] for token in custom_tokens)))